In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import geeksw.hgcal.testbeam as hgc
import scipy
from scipy.optimize import curve_fit
from scipy.special import erf

In [2]:
hdf = hgc.load_run(384)#, columns = ["beamEnergy","event", "rechit_layer", "rechit_energy", "rechit_X0", "rechit_dE"]
hdf.keys()
hdf.beamEnergy[0][0]

250.0

In [4]:
def grindhammer(t, alpha, beta, E):
    return E * ((beta*t)**(alpha-1)*beta*np.exp(-beta*t)) / scipy.special.gamma(alpha)
def f(x, mu, sigma, N):
    return N * np.exp(-(x - mu)**2/(2*sigma**2))
def fitting(X0, energy):
    return curve_fit(grindhammer, X0, energy)
def gaus(x,mu, sigma, A ):
    return A * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
def crystalball(x,n,a ,  xb, sig):
    x = x + 0j
    if a < 0:
        a = -a
    if n < 0:
        n = -n
    aa = abs(a)
    A = (n / aa) ** n * np.exp(-aa ** 2 / 2)
    B = n / aa - aa
    C = n / aa * 1 / (n - 1) * np.exp(-aa ** 2 / 2)
    D = np.sqrt(np.pi / 2) * (1 + erf(aa / np.sqrt(2)))
    N = 1.0 / (sig * (C + D))
    total = 0.0 * x
    total += ((x - xb) / sig > -a) * N * np.exp(-(x - xb) ** 2 / (2.0 * sig ** 2))
    total += ((x - xb) / sig <= -a) * N * A * (B - (x - xb) / sig) ** (-n)
    return total.real
    try:
        return total.real
    except:
        return total
    return total
def gausexp(x,  mu, sigma, N,k):
    if k < 0:
        k = -k

    total = 0.0 * x
    total += ((x - mu) / sigma > -k) * N * np.exp(-(x - mu) ** 2 / (2.0 * sigma ** 2))
    total += ((x - mu) / sigma <= -k) * N * np.exp(k ** 2 / 2.0 + k * ((x - mu) / sigma))

    return total
def p0gausexp(x, y, yerr):
    
    # Get mu
    mu_idx = np.argmax(y)
    mu = x[mu_idx]
    
    # get index ranges left and right of mu
    left_range = np.arange(len(x))[:mu_idx]
    right_range = np.arange(len(x))[mu_idx+1:]
    
    left_range = left_range[-len(right_range):]
    right_range = right_range[::-1]
    
    x_tail = x[left_range]
    y_left = y[left_range]
    y_right = y[right_range]
    yerr_left = yerr[left_range]
    yerr_right = yerr[right_range]
    
    k = (np.sum(np.sqrt((y_left - y_right)**2/(yerr_left*yerr_right)) < 1.) + 0.5) * (x[1] - x[0])
    
    N = y[mu_idx]
        
    x_mirror = np.concatenate([x[left_range], [mu], x[right_range]])
    y_mirror = np.concatenate([y[right_range], [y[mu_idx]], y[right_range]])
    
    sigma = np.sqrt(np.average((x_mirror - mu)**2, weights=y_mirror))
    k = k/sigma
    
    return [N, mu, sigma, k]

In [12]:
electron_runlist = hgc.runlist.query("Energy == '200'").query("Particle == 'electron'").query("Configuration == '22b'")#.apply(lambda x : x.iloc[1,2,3,4])
Resolution = []
run_numbers = electron_runlist.Run.values[:5]
for run in run_numbers:
    hdf = hgc.load_run(run, columns = ["beamEnergy","event", "rechit_layer", "rechit_energy", "rechit_X0", "rechit_dE"])
    rechit_energy_sum = hdf.groupby("event").rechit_energy.sum()
    rechit_energy_sum = rechit_energy_sum.reset_index()
    p = pd.DataFrame.quantile(rechit_energy_sum,[0.16,1.0])
    k1 = p.rechit_energy[0.16]
    k2 = p.rechit_energy[1.0]
    bins = np.linspace(k1, k2, 50)
    npoints, bin_edges = list(np.histogram(rechit_energy_sum.rechit_energy, bins=bins))
    bin_centers = (bin_edges[1:] + bin_edges[:-1])/2.
    bin_centers = bin_centers/np.median(bin_centers)
    npoints = npoints/np.sum(npoints)
    popt, pcov = curve_fit(gausexp, bin_centers, npoints, p0 = [0.77, 0.016, 0.17, 0.02], maxfev = 2000)#
    Resolution.append(popt[1]/popt[0])
    print (popt)
print (Resolution)
res_mean = np.abs(np.mean(Resolution))
res_variance = np.std(Resolution)


[ 0.68952927  0.01464439  0.36334996 -0.77736226]
[ 0.68786616  0.01477036  0.37076061 -0.87485475]
[ 0.68490967  0.01401201  0.37781203 -0.72032589]
[0.71843367 0.01489316 0.3240195  0.7231976 ]
[0.70121694 0.01385369 0.35824315 0.68677352]
[0.021238242845259905, 0.021472720091624226, 0.020458189320645165, 0.020730036919810894, 0.019756636742226368]


In [5]:
electron_runlist = hgc.runlist.query("Energy == '200'").query("Particle == 'electron'").query("Configuration == '22b'")
electron_runlist

,Run,Date,Nevents,Particle,Energy,Configuration,CaloConfiguration
372,664,2018-10-16 12:05:00,11000,electron,200,22b,1
373,665,2018-10-16 12:43:00,10239,electron,200,22b,1
374,666,2018-10-16 13:13:00,10534,electron,200,22b,1
375,667,2018-10-16 13:43:00,9930,electron,200,22b,1
378,671,2018-10-16 16:43:00,8982,electron,200,22b,1
379,672,2018-10-16 17:00:00,10167,electron,200,22b,1
380,673,2018-10-16 17:15:00,6136,electron,200,22b,1
381,674,2018-10-16 17:31:00,10111,electron,200,22b,1
382,675,2018-10-16 17:58:00,10421,electron,200,22b,1
383,676,2018-10-16 18:00:00,10094,electron,200,22b,1


In [12]:
Resolution = []
hdf = hgc.load_run(664, columns = ["beamEnergy","event", "rechit_layer", "rechit_energy", "rechit_X0", "rechit_dE"])
rechit_energy_sum = hdf.groupby("event").rechit_energy.sum()
rechit_energy_sum = rechit_energy_sum.reset_index()
p = pd.DataFrame.quantile(rechit_energy_sum,[0.16,1.0])
k1 = p.rechit_energy[0.16]
k2 = p.rechit_energy[1.0]
bins = np.linspace(k1, k2, 50)
npoints, bin_edges = list(np.histogram(rechit_energy_sum.rechit_energy, bins=bins))
bin_centers = (bin_edges[1:] + bin_edges[:-1])/2.
bin_centers = bin_centers/np.median(bin_centers)
npoints_variance = np.std(npoints)
npoints = npoints/np.sum(npoints)

print (npoints)
print (npoints_variance)

p0 = p0gausexp(bin_centers, npoints, npoints_variance)
print (p0)
popt, pcov = curve_fit(gausexp, bin_centers, npoints, p0, maxfev = 200000)#
#if (popt[0]>0.85):
    #popt, pcov = curve_fit(gausexp, bin_centers, npoints, p0 = [0.95, 0.02, 0.17, 0.20], maxfev = 20000)
Resolution.append(popt[1]/popt[0])
print (popt)
print (Resolution)


[8.51040880e-02 2.01164923e-01 3.47751052e-01 2.80767986e-01
 7.82008413e-02 6.36393054e-03 1.07863229e-04 1.07863229e-04
 0.00000000e+00 0.00000000e+00 1.07863229e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.07863229e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.07863229e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.07863229e-04]
639.6033841084114


IndexError: invalid index to scalar variable.

In [17]:
electron_runlist = hgc.runlist.query("Energy == '200'").query("Particle == 'electron'").query("Configuration == '22b'")#.apply(lambda x : x.iloc[1,2,3,4])
Resolution1 = []
run_numbers = electron_runlist.Run.values#[:2]
for run in run_numbers:
    hdf = hgc.load_run(run, columns = ["beamEnergy","event", "rechit_layer", "rechit_energy", "rechit_X0", "rechit_dE"])
    rechit_energy_sum = hdf.groupby("event").rechit_energy.sum()
    rechit_energy_sum = rechit_energy_sum.reset_index()
    p = pd.DataFrame.quantile(rechit_energy_sum,[0.16,1.0])
    k1 = p.rechit_energy[0.16]
    k2 = p.rechit_energy[1.0]
    bins = np.linspace(k1, k2, 50)
    npoints, bin_edges = list(np.histogram(rechit_energy_sum.rechit_energy, bins=bins))
    bin_centers = (bin_edges[1:] + bin_edges[:-1])/2.
    bin_centers = bin_centers/np.median(bin_centers)
    npoints = npoints/np.sum(npoints)
    popt, pcov = curve_fit(gausexp, bin_centers, npoints, p0 = [0.85, 0.02, 0.09,  1.12874385e-08], maxfev = 20000)#
    Resolution1.append(popt[1]/popt[0])
    print (popt)
print (Resolution1)
res_mean = np.abs(np.mean(Resolution1))
res_variance = np.std(Resolution1)


[8.39197401e-01 1.65996710e-02 7.77839727e-02 7.58992570e-09]
[ 8.39727540e-01  1.65463686e-02  7.84496495e-02 -7.86526833e-09]
[ 8.37117617e-01  1.59650460e-02  7.64382265e-02 -8.52780478e-09]
[ 8.37815670e-01  1.47628153e-02  8.07069743e-02 -1.07609807e-09]
[8.20426885e-01 8.04352083e-03 7.90975868e-02 2.24367123e-09]
[ 8.39094940e-01  1.63039920e-02  7.79843800e-02 -2.25266531e-08]
[ 8.36244925e-01  1.61875624e-02  7.54527322e-02 -1.23306689e-09]
[8.39572612e-01 1.65370959e-02 7.82770233e-02 1.93308361e-08]
[ 8.38856209e-01  1.55435209e-02  8.01469808e-02 -2.26563004e-09]
[ 8.34182316e-01  1.71875694e-02  7.28690967e-02 -4.02077278e-10]
[0.019780412828024316, 0.01970444909838634, 0.019071449012017638, 0.017620600624716265, 0.009804067830260202, 0.019430449620038977, 0.019357441721204054, 0.019697040639179576, 0.018529422281021814, 0.02060409226274131]


In [18]:
electron_runlist = hgc.runlist.query("Energy == '20'").query("Particle == 'electron'").query("Configuration == '22b'")#.apply(lambda x : x.iloc[1,2,3,4])
Resolution2 = []
run_numbers = electron_runlist.Run.values#[:2]
for run in run_numbers:
    hdf = hgc.load_run(run, columns = ["beamEnergy","event", "rechit_layer", "rechit_energy", "rechit_X0", "rechit_dE"])
    rechit_energy_sum = hdf.groupby("event").rechit_energy.sum()
    rechit_energy_sum = rechit_energy_sum.reset_index()
    p = pd.DataFrame.quantile(rechit_energy_sum,[0.16,1.0])
    k1 = p.rechit_energy[0.16]
    k2 = p.rechit_energy[1.0]
    bins = np.linspace(k1, k2, 50)
    npoints, bin_edges = list(np.histogram(rechit_energy_sum.rechit_energy, bins=bins))
    bin_centers = (bin_edges[1:] + bin_edges[:-1])/2.
    bin_centers = bin_centers/np.median(bin_centers)
    npoints = npoints/np.sum(npoints)
    popt, pcov = curve_fit(gausexp, bin_centers, npoints, p0 = [0.77, 0.02, 0.17, 0.20], maxfev = 20000)#
    if (popt[0]>0.85):
        popt, pcov = curve_fit(gausexp, bin_centers, npoints, p0 = [0.95, 0.02, 0.17, 0.20], maxfev = 20000)
    Resolution2.append(popt[1]/popt[0])
    print (popt)
print (Resolution2)
res_mean = np.abs(np.mean(Resolution2))
res_variance = np.std(Resolution2)


[0.9784569  0.0501377  0.06711682 0.43803248]
[0.75723526 0.04250836 0.13294887 0.54733639]


/opt/exp_soft/llr/python/3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: overflow encountered in exp
/opt/exp_soft/llr/python/3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in multiply


[1.03559916 0.05740707 0.05158031 0.50759264]
[0.76169845 0.04151949 0.12978478 0.48924957]
[0.76512649 0.04235938 0.13027091 0.51830376]
[0.76907353 0.04123552 0.12895013 0.48339668]
[0.73523129 0.04008638 0.14158905 0.50226989]
[0.71527871 0.03728964 0.15093927 0.44754142]
[0.76358171 0.04099566 0.12890474 0.47686733]
[0.82472534 0.04532435 0.10915475 0.50942337]
[0.74325489 0.0390763  0.14250013 0.48836612]
[0.92482253 0.04984561 0.07953175 0.47979819]
[1.02289956 0.05395035 0.05861683 0.49620009]
[0.05124160238370631, 0.056136266199292746, 0.055433681732696485, 0.054509097188084284, 0.05536259052057309, 0.05361713185096805, 0.05452213818829027, 0.052133018396544134, 0.05368863214354959, 0.05495690239538106, 0.052574559143028235, 0.05389749020849102, 0.05274256417029962]


In [19]:
df = pd.DataFrame({
    "Resolution": Resolution,
    "Resolution1": Resolution1,
    "Resolution2": Resolution2
})
df

,Resolution,Resolution1,Resolution2
0,0.074127,0.051241,0.051242
1,0.056136,0.056136,0.056136
2,0.055434,0.055434,0.055434
3,0.054509,0.003791,0.054509
4,0.055363,0.055363,0.055363
5,0.053617,-0.002819,0.053617
6,0.054522,-0.074128,0.054522
7,0.052133,0.052133,0.052133
8,0.053689,-0.003786,0.053689
9,0.054957,0.001145,0.054957


In [15]:
electron_runlist = hgc.runlist.query("Energy == '150'").query("Particle == 'electron'").query("Configuration == '22b'")
electron_runlist

,Run,Date,Nevents,Particle,Energy,Configuration,CaloConfiguration
231,493,2018-10-14 12:25:00,10672,electron,150,22b,1
232,494,2018-10-14 12:40:00,12220,electron,150,22b,1
233,495,2018-10-14 12:53:00,10544,electron,150,22b,1
234,496,2018-10-14 13:03:00,21977,electron,150,22b,1
235,501,2018-10-14 14:08:00,10499,electron,150,22b,1
236,502,2018-10-14 14:17:00,10728,electron,150,22b,1
237,503,2018-10-14 14:28:00,10018,electron,150,22b,1
238,504,2018-10-14 14:39:00,10933,electron,150,22b,1
239,505,2018-10-14 14:50:00,12459,electron,150,22b,1
240,506,2018-10-14 15:03:00,8800,electron,150,22b,1


In [38]:
Resolution = []
hdf = hgc.load_run(493, columns = ["beamEnergy","event", "rechit_layer", "rechit_energy", "rechit_X0", "rechit_dE"])
rechit_energy_sum = hdf.groupby("event").rechit_energy.sum()
rechit_energy_sum = rechit_energy_sum.reset_index()
p = pd.DataFrame.quantile(rechit_energy_sum,[0.16,1.0])
k1 = p.rechit_energy[0.16]
k2 = p.rechit_energy[1.0]
bins = np.linspace(k1, k2, 50)
npoints, bin_edges = list(np.histogram(rechit_energy_sum.rechit_energy, bins=bins))
bin_centers = (bin_edges[1:] + bin_edges[:-1])/2.
bin_centers = bin_centers/np.median(bin_centers)
npoints = npoints/np.sum(npoints)
popt, pcov = curve_fit(gausexp, bin_centers, npoints, p0 = [0.77, 0.02, 0.09, 0], maxfev = 20000)#
#if (popt[0]>0.85):
    #popt, pcov = curve_fit(gausexp, bin_centers, npoints, p0 = [0.95, 0.02, 0.17, 0.20], maxfev = 20000)
Resolution.append(popt[1]/popt[0])
print (popt)
print (Resolution)

[0.77 0.02 0.09 0.  ]
[0.025974025974025972]
